### Connect to Cognite with OIDC

CDF UI:
- https://hafslundeco.fusion.cognite.com/
- heco-dev

Cognite Python SDK:
- https://cognite-docs.readthedocs-hosted.com/projects/cognite-sdk-python/en/latest/
- `pip install cognite-sdk msal`


In [ ]:
import os
import pandas as pd
import numpy as np

from datetime import datetime
from msal import PublicClientApplication

from cognite.client import CogniteClient
from cognite.client.data_classes import TimeSeries, Asset

In [ ]:
# Log-in detaljer
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
CDF_CLUSTER = "az-power-no-northeurope"
COGNITE_PROJECT = "heco-dev"

In [ ]:
# Code to log-in WIHTOUT client_secret
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]

AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000


def authenticate_azure():

    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)

    # interactive login - make sure you have http://localhost:port in Redirect URI in App Registration as type "Mobile and desktop applications"
    creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)
    return creds


creds = authenticate_azure()

client = CogniteClient(
    token_url=creds["id_token_claims"]["iss"],
    token=creds["access_token"],
    token_client_id=creds["id_token_claims"]["aud"],
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    client_name="cognite-python-dev",
)

print(client.iam.token.inspect())

In [ ]:
client.time_series.list(limit=1)

### Generelle tips
- Sjekk at du er i riktig prosjekt: `heco-dev`
- Sjekk at du bruker riktig datasett: `uc:006:ml_test`
- Sjekk datamodell / assethierarki i GUI
    - Den store fordelen med CDF er at du koble sammen forskjellige datakilder og kontekstualisere dem, dvs. sette dem i sammenheng
- Husk å konvertere alle tidsstempler til UTC

### Playaround
- Create a new asset
- Create a new timeseries
- Write to timeseries
- Read data from timeseries

In [ ]:
# Create an asset
asset_name = "NO1_test"
client.assets.create([Asset(
    external_id=asset_name,
    name=asset_name,
    data_set_id=client.data_sets.retrieve(external_id="uc:006:ml_test").id,
)])

In [16]:
# Create a new timeseries connected to asset `NO1`
forecast_date = datetime(2022, 5, 20, 12, 0, 0, 0).replace(microsecond=0)
name = f"NO1_generation_test_series_{forecast_date}"

client.time_series.create(
    time_series=TimeSeries(
        external_id=name,
        asset_id=client.assets.retrieve(external_id="NO1_test").id,
        name=name,
        unit="MWh/h",
        data_set_id=client.data_sets.retrieve(external_id="uc:006:ml_test").id
        )
    )


,value
id,2661217891469451
external_id,NO1_generation_test_series_2022-05-20 12:00:00
name,NO1_generation_test_series_2022-05-20 12:00:00
is_string,False
unit,MWh/h
asset_id,2214652300307805
is_step,False
security_categories,[]
data_set_id,788450409833218
created_time,1655809271139


In [20]:
# Write datapoints to timeseries
# Tuple of datetime and value, remember time is given in UTC
datapoints = [
    (datetime(2022,6,20,9), 0),
    (datetime(2022,6,20,9,58), 5),
    (datetime(2022,6,20,10), 30),
    (datetime(2022,6,20,10,50), 24),
    (datetime(2022,6,20,11,50), 0),
]   

client.datapoints.insert(datapoints, id=client.time_series.retrieve(external_id="NO1_generation_test_series_2022-05-20 12:00:00").id)

In [22]:
# Get datapoints in pandas dataframe
df = client.datapoints.retrieve_dataframe(
    start=datetime(2022, 6, 20),
    end=datetime(2022, 6, 21),
    aggregates=["average", "sum"],
    granularity="1h",
    id=client.time_series.retrieve(external_id="NO1_generation_test_series_2022-05-20 12:00:00").id,
)
print(df)

                     2661217891469451|average  2661217891469451|sum
2022-06-20 09:00:00                       3.0                   5.0
2022-06-20 10:00:00                      27.0                  54.0
2022-06-20 11:00:00                       0.0                   0.0
